In [18]:
# ============================================================================
# NOTEBOOK 2 : NETTOYAGE DES DONNÉES E-COMMERCE
# ============================================================================
# Objectif : Nettoyer SYSTEMATIQUEMENT toutes les anomalies identifiées
# Pipeline : Automatisable et scalable
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🧹 NETTOYAGE SYSTÉMATIQUE DES DONNÉES E-COMMERCE")
print("=" * 55)
print("📋 Plan d'action basé sur l'analyse qualité")
print("🎯 Approche MLOps : fonctions réutilisables")


🧹 NETTOYAGE SYSTÉMATIQUE DES DONNÉES E-COMMERCE
📋 Plan d'action basé sur l'analyse qualité
🎯 Approche MLOps : fonctions réutilisables


In [19]:
# ============================================================================
# 📊 1. CHARGEMENT ET ÉTAT INITIAL
# ============================================================================

# Chargement des données
df = pd.read_csv('C:/Users/Moi/E-commerce_Marketing_Analytics/data/raw/online_retail_II.csv', encoding='iso-8859-1')

print("📊 ÉTAT INITIAL DU DATASET")
print("=" * 30)
print(f"Lignes : {len(df):,}")
print(f"Colonnes : {len(df.columns)}")
print(f"Mémoire : {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Snapshot des problèmes AVANT nettoyage
initial_issues = {
    'missing_customer_id': df['Customer ID'].isnull().sum(),
    'missing_description': df['Description'].isnull().sum(),
    'total_duplicates': df.duplicated().sum(),
    'negative_quantities': (df['Quantity'] < 0).sum(),
    'zero_prices': (df['Price'] == 0).sum(),
    'negative_prices': (df['Price'] < 0).sum(),
    'wrong_dtypes': 2  # InvoiceDate + Customer ID
}

print("\n🔴 PROBLÈMES IDENTIFIÉS :")
for issue, count in initial_issues.items():
    print(f"  {issue}: {count:,}")

total_issues = sum(initial_issues.values())
print(f"\n📊 TOTAL PROBLÈMES : {total_issues:,}")


📊 ÉTAT INITIAL DU DATASET
Lignes : 1,067,371
Colonnes : 8
Mémoire : 345.1 MB

🔴 PROBLÈMES IDENTIFIÉS :
  missing_customer_id: 243,007
  missing_description: 4,382
  total_duplicates: 34,335
  negative_quantities: 22,950
  zero_prices: 6,202
  negative_prices: 5
  wrong_dtypes: 2

📊 TOTAL PROBLÈMES : 310,883


In [ ]:
# ============================================================================
# 🔧 2. FONCTIONS DE NETTOYAGE (PIPELINE RÉUTILISABLE)
# ============================================================================

def clean_data_types(df):
    """
    Convertit les types de données aux formats appropriés
    NOMS RÉELS: Invoice, Price, Customer ID, InvoiceDate
    """
    df_clean = df.copy()
    
    print("🔧 CORRECTION DES TYPES DE DONNÉES")
    print("=" * 35)
    
    # 1. Conversion InvoiceDate
    try:
        df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
        print("✅ InvoiceDate: object → datetime64")
    except:
        print("⚠️  Erreur conversion InvoiceDate")
    
    # 2. Customer ID: float → string (IDENTIFIANT MÉTIER)
    df_clean['Customer ID'] = df_clean['Customer ID'].apply(
        lambda x: str(int(x)) if pd.notna(x) else x
    )
    print("✅ Customer ID: float64 → string (identifiant métier)")
    print(f"   Exemple: {df_clean['Customer ID'].dropna().head(3).tolist()}")
    
    return df_clean

def handle_missing_values(df):
    """
    Gère les valeurs manquantes selon la logique métier
    """
    df_clean = df.copy()
    
    print("\n🔍 GESTION DES VALEURS MANQUANTES")
    print("=" * 35)
    
    # Customer ID manquants : créer des IDs temporaires
    missing_customers = df_clean['Customer ID'].isnull()
    n_missing = missing_customers.sum()
    
    if n_missing > 0:
        # Créer des IDs temporaires uniques
        temp_ids = [f"GUEST_{i:06d}" for i in range(n_missing)]
        df_clean.loc[missing_customers, 'Customer ID'] = temp_ids
        print(f"✅ {n_missing:,} Customer ID manquants → IDs temporaires")
        print(f"   Exemple: {temp_ids[:3]}")
    
    # Description manquantes : garder comme NaN pour analyse
    missing_desc = df_clean['Description'].isnull().sum()
    if missing_desc > 0:
        print(f"📝 {missing_desc:,} descriptions manquantes conservées")
    
    return df_clean

def handle_duplicates(df):
    """
    Identifie et supprime les doublons
    """
    df_clean = df.copy()
    
    print("\n🔄 GESTION DES DOUBLONS")
    print("=" * 25)
    
    # Identifier les doublons
    duplicates = df_clean.duplicated()
    n_duplicates = duplicates.sum()
    
    if n_duplicates > 0:
        print(f"🔍 {n_duplicates:,} doublons identifiés")
        
        # Analyser les doublons avant suppression - NOM CORRECT: Invoice
        duplicate_invoices = df_clean[duplicates]['Invoice'].nunique()
        print(f"📋 Factures concernées: {duplicate_invoices:,}")
        
        # Supprimer les doublons
        df_clean = df_clean.drop_duplicates()
        print(f"✅ {n_duplicates:,} doublons supprimés")
    else:
        print("✅ Aucun doublon détecté")
    
    return df_clean

def handle_business_logic(df):
    """
    Applique les règles métier e-commerce
    """
    df_clean = df.copy()
    
    print("\n🏪 LOGIQUE MÉTIER E-COMMERCE")
    print("=" * 30)
    
    # 1. Séparer les ventes et les retours - NOM CORRECT: Invoice
    returns = df_clean['Invoice'].str.startswith('C', na=False)
    sales = ~returns
    
    n_returns = returns.sum()
    n_sales = sales.sum()
    
    print(f"📊 Ventes normales: {n_sales:,}")
    print(f"📊 Retours/Annulations: {n_returns:,}")
    
    # 2. Marquer le type de transaction
    df_clean['Transaction_Type'] = np.where(returns, 'RETURN', 'SALE')
    
    # 3. Calculer le montant total 
    df_clean['Total_Amount'] = df_clean['Quantity'] * df_clean['Price']
    
    # 4. Identifier les transactions suspectes 
    df_clean['Suspicious'] = (
        (df_clean['Price'] <= 0) | 
        (df_clean['Quantity'] == 0) |
        (df_clean['Total_Amount'] == 0)
    )
    
    suspicious_count = df_clean['Suspicious'].sum()
    print(f"🚨 Transactions suspectes: {suspicious_count:,}")
    
    return df_clean

def handle_outliers(df):
    """
    Identifie et marque les outliers (sans les supprimer)
    """
    df_clean = df.copy()
    
    print("\n📊 GESTION DES OUTLIERS")
    print("=" * 25)
    
    # Outliers pour les prix - NOM CORRECT: Price
    price_positive = df_clean[df_clean['Price'] > 0]['Price']
    if len(price_positive) > 0:
        Q1_price = price_positive.quantile(0.25)
        Q3_price = price_positive.quantile(0.75)
        IQR_price = Q3_price - Q1_price
        price_upper = Q3_price + 1.5 * IQR_price
        
        df_clean['Price_Outlier'] = (df_clean['Price'] > price_upper) & (df_clean['Price'] > 0)
        price_outliers = df_clean['Price_Outlier'].sum()
        print(f"📈 Outliers prix: {price_outliers:,} (seuil: {price_upper:.2f})")
    
    # Outliers pour les quantités
    qty_positive = df_clean[df_clean['Quantity'] > 0]['Quantity']
    if len(qty_positive) > 0:
        Q1_qty = qty_positive.quantile(0.25)
        Q3_qty = qty_positive.quantile(0.75)
        IQR_qty = Q3_qty - Q1_qty
        qty_upper = Q3_qty + 1.5 * IQR_qty
        
        df_clean['Quantity_Outlier'] = (df_clean['Quantity'] > qty_upper) & (df_clean['Quantity'] > 0)
        qty_outliers = df_clean['Quantity_Outlier'].sum()
        print(f"📦 Outliers quantité: {qty_outliers:,} (seuil: {qty_upper:.0f})")
    
    print("📝 Outliers conservés mais marqués pour analyse")
    
    return df_clean
print("🔧 FONCTIONS DE NETTOYAGE CRÉÉES")
print("✅ Prêtes pour pipeline automatisé")


🔧 FONCTIONS DE NETTOYAGE CRÉÉES
✅ Prêtes pour pipeline automatisé


In [21]:
# ============================================================================
# 🚀 3. PIPELINE DE NETTOYAGE COMPLET
# ============================================================================

def complete_cleaning_pipeline(df):
    """
    Pipeline complet de nettoyage des données
    """
    print("🚀 DÉMARRAGE DU PIPELINE DE NETTOYAGE")
    print("=" * 45)
    
    # Étape 1: Types de données
    df_clean = clean_data_types(df)
    
    # Étape 2: Valeurs manquantes
    df_clean = handle_missing_values(df_clean)
    
    # Étape 3: Doublons
    df_clean = handle_duplicates(df_clean)
    
    # Étape 4: Logique métier
    df_clean = handle_business_logic(df_clean)
    
    # Étape 5: Outliers
    df_clean = handle_outliers(df_clean)
    
    return df_clean

# Exécution du pipeline
print("🎯 EXÉCUTION DU PIPELINE COMPLET")
print("=" * 40)

df_cleaned = complete_cleaning_pipeline(df)

print(f"\n✅ NETTOYAGE TERMINÉ !")
print(f"📊 Données finales: {len(df_cleaned):,} lignes")
print(f"📊 Nouvelles colonnes: {len(df_cleaned.columns)}")


🎯 EXÉCUTION DU PIPELINE COMPLET
🚀 DÉMARRAGE DU PIPELINE DE NETTOYAGE
🔧 CORRECTION DES TYPES DE DONNÉES
✅ InvoiceDate: object → datetime64
✅ Customer ID: float64 → string (identifiant métier)
   Exemple: ['13085', '13085', '13085']

🔍 GESTION DES VALEURS MANQUANTES
✅ 243,007 Customer ID manquants → IDs temporaires
   Exemple: ['GUEST_000000', 'GUEST_000001', 'GUEST_000002']
📝 4,382 descriptions manquantes conservées

🔄 GESTION DES DOUBLONS
🔍 26,479 doublons identifiés
📋 Factures concernées: 5,132
✅ 26,479 doublons supprimés

🏪 LOGIQUE MÉTIER E-COMMERCE
📊 Ventes normales: 1,021,752
📊 Retours/Annulations: 19,140
🚨 Transactions suspectes: 6,206

📊 GESTION DES OUTLIERS
📈 Outliers prix: 67,113 (seuil: 8.50)
📦 Outliers quantité: 53,759 (seuil: 28)
📝 Outliers conservés mais marqués pour analyse

✅ NETTOYAGE TERMINÉ !
📊 Données finales: 1,040,892 lignes
📊 Nouvelles colonnes: 13


In [22]:
# ============================================================================
# 📊 ANALYSE DÉTAILLÉE DES RÉSULTATS DE NETTOYAGE
# ============================================================================

print("🎯 ANALYSE DÉTAILLÉE DES RÉSULTATS")
print("=" * 40)

# 1. Résumé des transformations
print("📈 RÉSUMÉ DES TRANSFORMATIONS :")
print(f"  • Lignes originales: {len(df):,}")
print(f"  • Lignes finales: {len(df_cleaned):,}")
print(f"  • Lignes supprimées: {len(df) - len(df_cleaned):,}")
print(f"  • Colonnes ajoutées: {len(df_cleaned.columns) - len(df.columns)}")

# 2. Analyse des nouvelles colonnes
print("\n🆕 NOUVELLES COLONNES CRÉÉES :")
new_columns = [col for col in df_cleaned.columns if col not in df.columns]
for col in new_columns:
    print(f"  • {col}")

# 3. Analyse des Customer ID
print("\n👥 ANALYSIS DES CUSTOMER ID :")
total_customers = df_cleaned['Customer ID'].nunique()
guest_customers = df_cleaned['Customer ID'].str.startswith('GUEST_').sum()
real_customers = total_customers - df_cleaned['Customer ID'].str.startswith('GUEST_').nunique()

print(f"  • Total clients uniques: {total_customers:,}")
print(f"  • Clients enregistrés: {real_customers:,}")
print(f"  • Clients invités: {df_cleaned['Customer ID'].str.startswith('GUEST_').nunique():,}")

# 4. Analyse des transactions
print("\n💰 ANALYSE DES TRANSACTIONS :")
transaction_summary = df_cleaned['Transaction_Type'].value_counts()
print(f"  • Ventes: {transaction_summary.get('SALE', 0):,}")
print(f"  • Retours: {transaction_summary.get('RETURN', 0):,}")

# 5. Analyse des montants
print("\n💵 ANALYSE DES MONTANTS :")
total_sales = df_cleaned[df_cleaned['Transaction_Type'] == 'SALE']['Total_Amount'].sum()
total_returns = df_cleaned[df_cleaned['Transaction_Type'] == 'RETURN']['Total_Amount'].sum()
net_revenue = total_sales + total_returns  # Les retours sont déjà négatifs

print(f"  • Chiffre d'affaires brut: ${total_sales:,.2f}")
print(f"  • Montant des retours: ${abs(total_returns):,.2f}")
print(f"  • Chiffre d'affaires net: ${net_revenue:,.2f}")

# 6. Analyse des outliers
print("\n🚨 ANALYSE DES OUTLIERS :")
price_outliers = df_cleaned['Price_Outlier'].sum()
qty_outliers = df_cleaned['Quantity_Outlier'].sum()
suspicious = df_cleaned['Suspicious'].sum()

print(f"  • Outliers prix: {price_outliers:,} ({price_outliers/len(df_cleaned)*100:.1f}%)")
print(f"  • Outliers quantité: {qty_outliers:,} ({qty_outliers/len(df_cleaned)*100:.1f}%)")
print(f"  • Transactions suspectes: {suspicious:,} ({suspicious/len(df_cleaned)*100:.1f}%)")

# 7. Aperçu des données nettoyées
print("\n📋 APERÇU DES DONNÉES NETTOYÉES :")
print(df_cleaned.head())


🎯 ANALYSE DÉTAILLÉE DES RÉSULTATS
📈 RÉSUMÉ DES TRANSFORMATIONS :
  • Lignes originales: 1,067,371
  • Lignes finales: 1,040,892
  • Lignes supprimées: 26,479
  • Colonnes ajoutées: 5

🆕 NOUVELLES COLONNES CRÉÉES :
  • Transaction_Type
  • Total_Amount
  • Suspicious
  • Price_Outlier
  • Quantity_Outlier

👥 ANALYSIS DES CUSTOMER ID :
  • Total clients uniques: 248,949
  • Clients enregistrés: 248,947
  • Clients invités: 2

💰 ANALYSE DES TRANSACTIONS :
  • Ventes: 1,021,752
  • Retours: 19,140

💵 ANALYSE DES MONTANTS :
  • Chiffre d'affaires brut: $20,445,293.52
  • Montant des retours: $1,516,344.05
  • Chiffre d'affaires net: $18,928,949.47

🚨 ANALYSE DES OUTLIERS :
  • Outliers prix: 67,113 (6.4%)
  • Outliers quantité: 53,759 (5.2%)
  • Transactions suspectes: 6,206 (0.6%)

📋 APERÇU DES DONNÉES NETTOYÉES :
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   

In [23]:
# ============================================================================
# 🔍 CONTRÔLE QUALITÉ FINAL
# ============================================================================

print("\n🔍 CONTRÔLE QUALITÉ FINAL")
print("=" * 30)

# 1. Vérification des valeurs manquantes
print("📊 VALEURS MANQUANTES RESTANTES :")
missing_summary = df_cleaned.isnull().sum()
for col, missing in missing_summary.items():
    if missing > 0:
        print(f"  • {col}: {missing:,} ({missing/len(df_cleaned)*100:.1f}%)")

# 2. Vérification des types de données
print("\n🔧 TYPES DE DONNÉES FINAUX :")
for col in df_cleaned.columns:
    print(f"  • {col}: {df_cleaned[col].dtype}")

# 3. Vérification des doublons
remaining_duplicates = df_cleaned.duplicated().sum()
print(f"\n🔄 DOUBLONS RESTANTS: {remaining_duplicates:,}")

# 4. Cohérence des données
print("\n✅ TESTS DE COHÉRENCE :")
print(f"  • Dates valides: {df_cleaned['InvoiceDate'].notna().sum():,}")
print(f"  • Customer IDs valides: {df_cleaned['Customer ID'].notna().sum():,}")
print(f"  • Montants calculés: {(df_cleaned['Total_Amount'] == df_cleaned['Quantity'] * df_cleaned['Price']).sum():,}")

print("\n🎉 CONTRÔLE QUALITÉ TERMINÉ !")



🔍 CONTRÔLE QUALITÉ FINAL
📊 VALEURS MANQUANTES RESTANTES :
  • Description: 4,382 (0.4%)

🔧 TYPES DE DONNÉES FINAUX :
  • Invoice: object
  • StockCode: object
  • Description: object
  • Quantity: int64
  • InvoiceDate: datetime64[ns]
  • Price: float64
  • Customer ID: object
  • Country: object
  • Transaction_Type: object
  • Total_Amount: float64
  • Suspicious: bool
  • Price_Outlier: bool
  • Quantity_Outlier: bool

🔄 DOUBLONS RESTANTS: 0

✅ TESTS DE COHÉRENCE :
  • Dates valides: 1,040,892
  • Customer IDs valides: 1,040,892
  • Montants calculés: 1,040,892

🎉 CONTRÔLE QUALITÉ TERMINÉ !


In [26]:
# ============================================================================
# 💾 SAUVEGARDE DES DONNÉES NETTOYÉES
# ============================================================================

print("💾 SAUVEGARDE DES DONNÉES NETTOYÉES")
print("=" * 35)

# Sauvegarder le dataset nettoyé
df_cleaned.to_pickle('C:/Users/Moi/E-commerce_Marketing_Analytics/data/processed/cleaned_ecommerce_data.pkl')
print("✅ Données sauvegardées: 'data/cleaned_ecommerce_data.csv'")

# Sauvegarder un résumé du nettoyage
cleaning_summary = {
    'original_rows': len(df),
    'final_rows': len(df_cleaned),
    'duplicates_removed': len(df) - len(df_cleaned),
    'guest_customers_created': df_cleaned['Customer ID'].str.startswith('GUEST_').sum(),
    'total_revenue': df_cleaned[df_cleaned['Transaction_Type'] == 'SALE']['Total_Amount'].sum(),
    'outliers_identified': df_cleaned['Price_Outlier'].sum() + df_cleaned['Quantity_Outlier'].sum()
}

print("\n📊 RÉSUMÉ DU NETTOYAGE :")
for key, value in cleaning_summary.items():
    print(f"  • {key}: {value:,}")

print("\n🎯 DONNÉES PRÊTES POUR L'ANALYSE !")



💾 SAUVEGARDE DES DONNÉES NETTOYÉES
✅ Données sauvegardées: 'data/cleaned_ecommerce_data.csv'

📊 RÉSUMÉ DU NETTOYAGE :
  • original_rows: 1,067,371
  • final_rows: 1,040,892
  • duplicates_removed: 26,479
  • guest_customers_created: 243,007
  • total_revenue: 20,445,293.518
  • outliers_identified: 120,872

🎯 DONNÉES PRÊTES POUR L'ANALYSE !
